In [11]:
import pandas as pd
import numpy as np

import os
import shutil

import sys, pathlib, fitz

from dateutil import parser

### Data Import

In [12]:
mint_transaction_history = pd.read_csv('sourceData/mint_2019_2024.csv')

mint_transaction_history.head(10)

mint_transaction_history['Account Name'].unique()

mint_account_aliases = {
    'TD BEYOND CHECKING': 'Checking [3773]',
    'TD CONVENIENCE CHECKING': 'Checking [7985]',
    'HYSA': 'American Express Savings',
    'Individual ...959': 'Charles Schwab Individual Investment Account',
    'CREDITCARD Account': 'Capital One Credit Card'
}

mint_transaction_history['Account Name'].replace(mint_account_aliases, inplace=True)

mint_transaction_history

,Date,Description,Original Description,Amount,Transaction Type,Category,Account Name,Labels,Notes
0,12/30/2023,LinkedIn,LINKEDIN PRE 975246241,42.49,debit,Career,Apple Card,NaN,NaN
1,12/29/2023,VERIS INSIGHTS L PAYROLL,VERIS INSIGHTS L PAYROLL,2315.42,credit,Paycheck,Checking [3773],NaN,NaN
2,12/29/2023,Blue Bottle Coffee,SQ *BLUE BOTTLE COFF,11.24,debit,Coffee Shops,Checking [3773],NaN,NaN
3,12/29/2023,VERIS INSIGHTS L PAYROLL,VERIS INSIGHTS L PAYROLL,408.60,credit,Paycheck,American Express Savings,NaN,NaN
4,12/29/2023,MBTA KENMORE,MBTA KENMORE,20.00,debit,Public Transportation,Apple Card,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7346,8/10/2018,Starbucks,469216 STARBUCKS STORE 00833469216 STA...,3.37,debit,Coffee Shops,Checking [3773],NaN,NaN
7347,8/10/2018,Sweetgreen Seaport Squ,469216 SWEETGREEN SEAPORT SQ469216 SWE...,13.32,debit,Restaurants,Checking [3773],NaN,NaN
7348,8/09/2018,Starbucks,469216 STARBUCKS STORE 00875469216 STA...,3.47,debit,Coffee Shops,Checking [3773],NaN,NaN
7349,8/08/2018,Starbucks,469216 STARBUCKS STORE 00875469216 STA...,2.94,debit,Coffee Shops,Checking [3773],NaN,NaN


### PDF (Original Balance) Handling

In [13]:
def find_balance(statement_path):
    
    full_doc = fitz.open(statement_path)
    cover_page = full_doc[0]
    words = cover_page.get_text("words", sort=True)

    for i, word_obj in enumerate(words):

        text = word_obj[4]

        if text == 'Ending':

            if words[i + 1][4] == 'Balance':

                end_bal = words[i + 2][4]

                print(text)
                print(words[i + 1][4])
                print(words[i + 2][4])

        if text == 'Statement':

            if words[i + 1][4] == 'Period:':

                end_month = words[i + 4][4].split('-')[1]
                end_day = words[i + 5][4]
                end_year = words[i + 6][4]

                full_statement_end_str = end_month + "-" + end_day + "-" + end_year

                print(text)
                print(words[i + 1][4])
                print(f"{end_month} {end_day} {end_year}")

    return end_bal, full_statement_end_str

original_balance_statement_paths = ['sourceData/OriginalBalance_PDF_9_2018_3773.pdf', 'sourceData/OriginalBalance_PDF_9_2018_7985.pdf']
original_statement_info = {}

for statement_path in original_balance_statement_paths:

    statement_info = find_balance(statement_path)

    account_id = statement_path.split('.')[0][-4:]

    end_bal = statement_info[0]
    full_statement_end_str = statement_info[1]

    original_statement_info[account_id] = [end_bal, full_statement_end_str]

original_statement_info

Statement
Period:
Sep 07 2018
Ending
Balance
2,439.91
Statement
Period:
Sep 11 2018
Ending
Balance
6,551.34


{'3773': ['2,439.91', 'Sep-07-2018'], '7985': ['6,551.34', 'Sep-11-2018']}

### CSV (Mint Transaction History) Handling

In [14]:
mint_transaction_history['Date'] = pd.to_datetime(mint_transaction_history['Date'], format='mixed', dayfirst=False, yearfirst=False)

In [15]:
accounts_end_dates = {str(key): value[1] for key, value in original_statement_info.items()}

for account in accounts_end_dates:

    print(account)
    print(accounts_end_dates[account])

    mint_transaction_history = mint_transaction_history[((mint_transaction_history['Date'] > accounts_end_dates[account]) & (mint_transaction_history['Account Name'].str.contains(account))) | (~mint_transaction_history['Account Name'].str.contains(account))]

original_balance = sum([float(value[0].replace(',', '')) for key, value in original_statement_info.items()])

original_balance

3773
Sep-07-2018
7985
Sep-11-2018


8991.25

In [16]:
mint_transaction_history.loc[mint_transaction_history['Transaction Type'] == 'debit', 'Amount'] *= -1

## -- Best Estimate De-Duplication -- ##

duplicates = mint_transaction_history[mint_transaction_history.duplicated(['Transaction Type', 'Account Name', 'Date', 'Amount'], keep=False)]

two_instance_duplicates = duplicates.groupby(['Transaction Type', 'Account Name', 'Date', 'Amount']).size()

two_instance_duplicates = two_instance_duplicates[two_instance_duplicates == 2]

indicies_to_remove = mint_transaction_history[mint_transaction_history[['Transaction Type', 'Account Name', 'Date', 'Amount']].apply(tuple, axis=1).isin(two_instance_duplicates.index)].index[::2]

mint_transaction_history.drop(indicies_to_remove, inplace=True)

# ## -- Account Testing -- ##

account_3773 = mint_transaction_history[mint_transaction_history['Account Name'].str.contains('3773')]

initial_bal = float(original_statement_info['3773'][0].replace(',', ''))

account_3773 = account_3773.sort_values('Date', ascending=True)

account_3773['Cumulated Sum'] = initial_bal + account_3773['Amount'].cumsum()

account_3773.to_csv('account_3773_cumsum.csv')

### Checking Account Value Parsing

In [23]:
print(os.listdir('sourceData/checkingStatements'))

def cataloge_checking_history(statement):

    full_doc = fitz.open(statement)
    first_page = full_doc[0]
    all_words = first_page.get_text("words", sort=True)

    for word_index, word_obj in enumerate(all_words):
        
        word_text = word_obj[4]

        if word_text == 'Beginning': 

            if all_words[word_index + 1][4] == 'Balance':

                beginning_balance = all_words[word_index + 2][4]

        if word_text == 'Ending':

            if all_words[word_index + 1][4] == 'Balance':

                end_balance = all_words[word_index + 2][4]

        if word_text == 'Statement':

            if all_words[word_index + 1][4] == 'Period:':

                start_month = all_words[word_index + 2][4]
                start_day = all_words[word_index + 3][4]
                middle_string = all_words[word_index + 4][4]
                end_day = all_words[word_index + 5][4]
                end_year = all_words[word_index + 6][4]

                start_year = middle_string.split('-')[0]
                end_month = middle_string.split('-')[1]

                print(f'{start_day}/{start_month}/{start_year} -> {end_day}/{end_month}/{end_year}')

    print(beginning_balance)
    print(end_balance)

    return all_words

all_words = cataloge_checking_history(statement_path)

['View PDF Statement_2021-05-07.pdf', 'View PDF Statement_2021-09-07.pdf', 'View PDF Statement_2022-10-07.pdf', 'View PDF Statement_2022-02-07.pdf', 'View PDF Statement_2023-04-07.pdf', 'View PDF Statement_2023-08-07.pdf', 'View PDF Statement_2019-05-07.pdf', 'View PDF Statement_2020-11-07.pdf', 'View PDF Statement_2020-03-07.pdf', 'View PDF Statement_2019-09-07.pdf', 'View PDF Statement_2024-02-07.pdf', 'View PDF Statement_2023-06-07.pdf', 'View PDF Statement_2020-01-07.pdf', 'View PDF Statement_2019-07-07.pdf', 'View PDF Statement_2021-07-07.pdf', 'View PDF Statement_2022-12-07.pdf', 'View PDF Statement_2020-09-07.pdf', 'View PDF Statement_2019-03-07.pdf', 'View PDF Statement_2020-05-07.pdf', 'View PDF Statement_2019-11-07.pdf', 'View PDF Statement_2023-02-07.pdf', 'View PDF Statement_2023-10-07.pdf', 'View PDF Statement_2022-08-07.pdf', 'View PDF Statement_2022-04-07.pdf', 'View PDF Statement_2021-03-07.pdf', 'View PDF Statement_2021-11-07.pdf', 'View PDF Statement_2022-06-07.pdf', 

def find_balance(statement_path):
    
    full_doc = fitz.open(statement_path)
    cover_page = full_doc[0]
    words = cover_page.get_text("words", sort=True)

    for i, word_obj in enumerate(words):

        text = word_obj[4]

        if text == 'Ending':

            if words[i + 1][4] == 'Balance':

                end_bal = words[i + 2][4]

                print(text)
                print(words[i + 1][4])
                print(words[i + 2][4])

        if text == 'Statement':

            if words[i + 1][4] == 'Period:':

                end_month = words[i + 4][4].split('-')[1]
                end_day = words[i + 5][4]
                end_year = words[i + 6][4]

                full_statement_end_str = end_month + "-" + end_day + "-" + end_year

                print(text)
                print(words[i + 1][4])
                print(f"{end_month} {end_day} {end_year}")

    return end_bal, full_statement_end_str

original_balance_statement_paths = ['sourceData/OriginalBalance_PDF_9_2018_3773.pdf', 'sourceData/OriginalBalance_PDF_9_2018_7985.pdf']
original_statement_info = {}

for statement_path in original_balance_statement_paths:

    statement_info = find_balance(statement_path)

    account_id = statement_path.split('.')[0][-4:]

    end_bal = statement_info[0]
    full_statement_end_str = statement_info[1]

    original_statement_info[account_id] = [end_bal, full_statement_end_str]

original_statement_info